In [7]:
#
# Script to create a CSV file for the cats vs dogs dataset
#
import csv
import os

# Directory paths
cats_dir = "../data/train/cnn_train_images/cats"
dogs_dir = "../data/train/cnn_train_images/dogs"
csv_dir = '../data/train/cnn_train_images'

# File path within the folder
csv_file_path = os.path.join(csv_dir, "cats_vs_dogs_train.csv")

# List to store image paths and labels
data = []

# Process cats images
for filename in os.listdir(cats_dir):
    filepath = os.path.join(cats_dir, filename)
    
    # Splitting the string at the first occurrence of '.'
    id_number = filename.split('.', 1)

    # Keeping only the part before the first '.'
    id = id_number[0]

    data.append([id, 1, 0])  # Label 1 for cat

# Process dogs images
for filename in os.listdir(dogs_dir):
    filepath = os.path.join(dogs_dir, filename)
            
    # Splitting the string at the first occurrence of '.'
    id_number = filename.split('.', 1)

    # Keeping only the part before the first '.'
    id = id_number[0]

    data.append([id, 0, 1])  # Label 0 for non-hotdog

# Shuffle the data
import random
random.shuffle(data)

# Write data to CSV file
with open(csv_file_path, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Id", "Cat", "Dog"])
    csvwriter.writerows(data)


In [8]:
#
# Append labels from one CSV file to another (combine other train labels with cats vs dogs labels)
#

# Function to read CSV file and return a dictionary where keys are IDs
def read_csv_to_dict(file_path):
    data_dict = {}
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_dict[row['Id']] = row
    return data_dict

# Paths to the CSV files
csv1_path = "../data/train/train.csv"
csv2_path = "../data/train/cats_vs_dogs_train.csv"

# Read CSV files into dictionaries
csv1_data = read_csv_to_dict(csv1_path)
csv2_data = read_csv_to_dict(csv2_path)

# Append labels from CSV1 to CSV2 if IDs match
for id, row in csv2_data.items():
    if id in csv1_data:
        csv2_data[id].update(csv1_data[id])

# Write combined data to a new CSV file
combined_csv_path = "../data/train/cats_vs_dogs_train.csv"
with open(combined_csv_path, "w", newline='') as csvfile:
    fieldnames = list(csv2_data.values())[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(csv2_data.values())


In [31]:
import torch
import torchvision
import os
from torchvision import transforms, datasets
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Step 1: Prepare the Dataset
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0]) + ".jpg"
        image = Image.open(img_name)
        label = self.data.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label
    
# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load dataset
train_dataset = CustomDataset(csv_file='../data/train/cnn_train_images/cats_vs_dogs_train.csv',
                              root_dir='../data/train/cnn_train_images/',
                              transform=transform)

In [32]:
# Step 2: Define the CNN Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.fc1 = nn.Linear(128 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 2)  # 2 classes: Cat and Dog

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 128 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [36]:
# Step 3: Train the Model

# WINDOWS: Device will determine whether to run the training on GPU or CPU.
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MAC: Device will determine whether to run the training on GPU or CPU.
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

print('Finished Training')


Epoch [1/100], Loss: 0.6937
Epoch [2/100], Loss: 0.6779
Epoch [3/100], Loss: 0.7166
Epoch [4/100], Loss: 0.6884
Epoch [5/100], Loss: 0.6987
Epoch [6/100], Loss: 0.6912
Epoch [7/100], Loss: 0.6879
Epoch [8/100], Loss: 0.6931
Epoch [9/100], Loss: 0.7325
Epoch [10/100], Loss: 0.6979
Epoch [11/100], Loss: 0.6731
Epoch [12/100], Loss: 0.6342
Epoch [13/100], Loss: 0.6120
Epoch [14/100], Loss: 0.5294
Epoch [15/100], Loss: 0.4541
Epoch [16/100], Loss: 0.5888
Epoch [17/100], Loss: 0.4367
Epoch [18/100], Loss: 0.3118
Epoch [19/100], Loss: 0.2876
Epoch [20/100], Loss: 0.5139
Epoch [21/100], Loss: 0.3416
Epoch [22/100], Loss: 0.9613
Epoch [23/100], Loss: 0.8490
Epoch [24/100], Loss: 0.3861
Epoch [25/100], Loss: 0.3461
Epoch [26/100], Loss: 0.2692
Epoch [27/100], Loss: 0.7558
Epoch [28/100], Loss: 0.3385
Epoch [29/100], Loss: 0.2045
Epoch [30/100], Loss: 0.1172
Epoch [31/100], Loss: 0.1373
Epoch [32/100], Loss: 0.1479
Epoch [33/100], Loss: 0.0784
Epoch [34/100], Loss: 0.0241
Epoch [35/100], Loss: 0